In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures  # numeric값 곱함
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

* EDA

Data at first sight
Here is an exception for the data description for the competition
Features that belong to similar group

In [ ]:
# 디버깅
DEBUG = False
if DEBUG:
    NROWS = 500000
else:
    NROWS = None

In [ ]:
# 샘플로 EDA
%time
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv', nrows=NROWS)
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv', nrows=NROWS)
# train = train.sample(frac=0.1)

In [ ]:
train.head()
train.shape

In [ ]:
# balance 샘플 추출
from sklearn.model_selection import StratifiedKFold
fold = StratifiedKFold(n_splits=10, random_state=1980)

for trn_idx, val_idx in fold.split(train, train['target']):
    break

train = train.iloc[trn_idx]
train

In [ ]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')

In [ ]:
cat_cols = [col for col in train.columns if 'cat' in col]
cat_cols

In [ ]:
for col in cat_cols:
    print(col, train[col].nunique())  # 104개: high cardinality

In [ ]:
train.drop_duplicates()
train.shape
#train.info()

* Metadata
- To facilitate the data management, we'll store meta-information about the variables in a dataframe.
- This will be helpful when we want to select specific variables for analysis visualization, modeling.

Concretely, we will store
- role: input, ID, target
- level: nominal, interval, ordinal, binary
- keep: True or False
- dtype: int, float, str

In [ ]:
data = []
for f in train.columns:
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
        
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    keep = True
    if f == 'id':
        keep = False
    dtype = train[f].dtype
    
    f_dict = {
        'varname' : f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)

In [ ]:
meta = pd.DataFrame(data, )

In [ ]:
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])

In [ ]:
meta.set_index('varname', inplace=True)

In [ ]:
#meta

Example to extract all nominal variables that are not dropped

In [ ]:
meta.loc[(meta.level=='nominal') & (meta.keep)].index

- Below the number of variables perrole and level are displayed

In [ ]:
pd.DataFrame({'count':meta.groupby(['role', 'level'])['role'].size()}).reset_index()

In [ ]:
meta.groupby(['role', 'level'])['role'].size()

- Descriptive statistics
- We can also apply the describe method on the dataframe. 
- However, it doesn't make such sense to calculate the mean, std,.. on categorical variables and the id variable.
- We'll explore the categorical variables visually later.
- Thanks to our meta file we can easily select the variables on which want to compute the descriptive statistics. To keep things clear, we'll do this per data type.

In [ ]:
categorical_feats = [col for col in train.columns if 'cat' in col] 

In [ ]:
categorical_feats

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index

In [ ]:
train[v].describe()

<b> reg variables </b>
- only ps_reg_03 has missing values
- the range(min to max) differs betwwen the variables. We could appy scaling(e.g. StandardScaler), but it depends on the classifier we will want to use.

<b> car variables </b>
- ps_car_12 and ps_car_15 have missing values.
- again, the range differs and we could apply scaling.

<b> calc variables </b>
- no missing values
- this seems to be some kind of ratio as the maximum is 0.9
- all three _calc variables have very similar distributions.

<b> Overall </b>
we can see that the range of the interval variables is rather small. Perhaps some transformation (e.g.log) is already applied in order to anonymize the data?

## Ordinal variables

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

## Binary variables

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

- A priori in the train data is 3.645%, which is strongly imbalanced.
- From the means we can conclude that for most variables the value is zero in most cases.

## Handling imbalanced classes
As we mentioned above the proportion of records with target=1 is far less than target=0. This can lead to a model that has great accuracy but does have any added value in practice. Two possible strategies to deal with this problem are:

- oversampling records with target=1
- undersampling records with target=0

There are many more strategies of course and MachineLearningMastery.com gives a nice overview. As we have a rather large training set, we can go for <b> undersampling </b>.

In [ ]:
desired_apriori = 0.1
idx_0 = train[train.target==0].index
idx_1 = train[train.target==1].index

In [ ]:
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

In [ ]:
# Calcualate the undersampling rate and resulting number of records with target = 0
undersampling_rate = ((1 - desired_apriori) * nb_1) / (nb_0 * desired_apriori)
undersampled_nb_0 = int(undersampling_rate * nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

In [ ]:
train = train.loc[idx_list].reset_index(drop=True)
train

## Data Quality Checks

#### Checking missing values
Missings are represented as -1

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train.loc[train[f] == -1][f].count()  # loc 생략 가능
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

- ps_car_03_cat and ps_car_05_cat have a large proportion of records with missing values. Remove these variables.
- For the other categorical variables with missing values, we can leave the missing value -1 as such.
- ps_reg_03 (continuous) has missing values for 18% of all records. Replace by the mean.
- ps_car_11 (ordinal) has only 5 records with misisng values. Replace by the mode.
- ps_car_12 (continuous) has only 1 records with missing value. Replace by the mean.
- ps_car_14 (continuous) has missing values for 7% of all records. Replace by the mean.

In [ ]:
# Dropping the variables with too many missing values

vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, axis=1, inplace=True)
meta.loc[(vars_to_drop), 'keep'] = False  # Updating the meta

In [ ]:
temp_series = train[['ps_car_01_cat', 'ps_car_02_cat', 'ps_reg_03']].groupby(['ps_car_01_cat', 'ps_car_02_cat']).mean()
temp_series.reset_index(inplace=True)

In [ ]:
#train.loc[train['ps_reg_03'] == -1].merge(temp_series, on=['ps_car_01_cat', 'ps_car_02_cat'], how='left', prefix=['old', 'new'])

In [ ]:
# Imputing with the mean or mode
mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

In [ ]:
# ravel: 2차원 벡터 풀어줌
train['ps_reg_03'] =  mean_imp.fit_transform(train[['ps_reg_03']]).ravel()

Cardinality: 특정 feature에 얼마나 다른 데이터가 있는지
- Checking the cardinality of the categorical variables
- Cardinality refers to the number of different values in a variable. 
- As we will create dummy variables from the categorical variables later on, 
- we need to check whether there are variables with many distinct values. 
- We should handle these variables differently as they would result in many dummy variables.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

In [ ]:
trn_series = train['ps_car_11_cat']
tst_series = test['ps_car_11_cat']
target = train.target
min_samples_leaf = 100
smoothing = 10
noise_level = 0.01

In [ ]:
assert len(trn_series) == len(target)

In [ ]:
assert trn_series.name == tst_series.name

In [ ]:
temp = pd.concat([trn_series, target], axis=1)

In [ ]:
average = temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count'])
average

In [ ]:
temp.groupby(by=trn_series.name)[target.name].mean()

In [ ]:
# mean encoding
# 중요한 카테고리로 group by 후 각 카테고리의 통계값으로 피처 생성
def max_min(x):
    return x.max() - x.min()

temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count', max_min])

In [ ]:
# Compute smoothing: 오버피팅 없이 깔끔하게 곡선 그리기 위함
smoothing = 1 / (1 + np.exp(-(average['count'] - min_samples_leaf) / smoothing))

In [ ]:
prior = target.mean()

In [ ]:
average[target.name] = prior * (1 - smoothing) + average['mean'] * smoothing
average

In [ ]:
average.drop(['mean','count'], axis=1, inplace=True)
average

In [ ]:
trn_series.to_frame

In [ ]:
average.reset_index().rename(columns={'index':target.name, target.name:'average'})

In [ ]:
# 붙여질 데이터를 먼저 넣고 -> 붙일거를 넣고 -> 이름 바꾸고 -> 결합 기준 작성
ft_trn_series =  pd.merge(trn_series.to_frame(trn_series.name), 
                 average.reset_index().rename(columns={'index':target.name, target.name:'average'}),
                 on = trn_series.name, 
                 how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)

ft_trn_series.index = trn_series.index

In [ ]:
# train에서 얻은 averages를 test에 붙임
# train 카테고리와 test 카테고리가 같기 때문
ft_tst_series =  pd.merge(tst_series.to_frame(tst_series.name), 
                 average.reset_index().rename(columns={'index':target.name, target.name:'average'}),
                 on = tst_series.name, 
                 how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)

ft_tst_series.index = tst_series.index

In [ ]:
# 너무 분리된 값이라 노이즈 추가
# 노이즈레벨을 0.01로 넣으면 이정도 노이즈 생긴다는 것을 확인
# 너무 분리되지 않고 연속된 값을 가질 수 있도록 mean encoding

0.01 * np.random.randn(ft_trn_series.shape[0])

In [ ]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train.head()

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)

In [ ]:
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)

In [ ]:
meta.loc['ps_car_11_cat', 'keep'] = False

In [ ]:
meta

In [ ]:
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1,inplace=True)

## Mean target encoding
- 카테고리가 정말 많을 때 사용. 높은 카디널리티 카테고리에 유용
- 카테고리면 선형 관계 사용X. 숫자적인 비교가 가능하기 때문에 선형 관계가 생김
- 라벨 인코딩보다 더 적은 분류, 더 빠르게 배움

- 만들고 확인하기 어려움
- 정규화하지 않으면 overfit 되기 쉬움(정규화 필요)
- 인코딩 참고: https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

# Exploratory Data Visualization

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
v

In [ ]:
for f in v:
    break

In [ ]:
plt.figure()
fig, ax = plt.subplots(figsize=(20,10))
cat_perc = train[[f, 'target']].groupby([f], as_index=False).mean()

In [ ]:
cat_perc

In [ ]:
cat_perc.sort_values(by='target', ascending=False, inplace=True)

In [ ]:
sns.barplot(x=f, y='target', data=cat_perc, order=cat_perc[f])
plt.ylabel('% target')
plt.xlabel(f)

In [ ]:
sns.set(font_scale=1)

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20,10))
    # Calculate the percentage of target=1 per category value
    cat_perc = train[[f, 'target']].groupby([f],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target')
    plt.xlabel(f)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.show();

In [ ]:
# 그래프는 결과를 왜곡하기 쉽기 때문에 카운트도 확인(그래프에도 표시)
f = 'ps_car_02_cat'
cat_perc = train[[f,'target']].groupby([f], as_index=False).agg(['mean','count'])
cat_perc

- As we can see from the variables <b> with missing values </b>, it is a good idea to keep the missing values as a separate category value, instead of replacing them by the mode for instance. 
- The customers with a missing value appear to have a much higher (in some cases much lower) probability to ask for an insurance claim.

## Interval variables
- Checking the correlations between interval variables, A heatmap is a good way to visualize the correlation between variables. The code below is based on <b> an example by Michael Waskom </b>

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index

In [ ]:
correlations = train[v].corr()
#correlations

In [ ]:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmap

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
            square=True, linewidths=.5, annot=True, cbar_kws={'shrink':.5})

In [ ]:
# 한번에
def corr_heatmap(v):
    correlations = train[v].corr()

    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();
    
# v = meta[(meta.level == 'interval') & (meta.keep)].index
# corr_heatmap(v)

- There are a string correlations between the variables

In [ ]:
s = train.sample(frac=0.1)

In [ ]:
# lmplot: regression plot
sns.lmplot(x='ps_reg_02', y='ps_reg_03', 
           data=s, 
           hue='target', 
           palette='Set1', 
           scatter_kws={'alpha':0.3})  # alpha: 투명도
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', 
           data=s, 
           hue='target', 
           palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', 
           data=s, 
           hue='target', 
           palette='Set1', 
           scatter_kws={'alpha':0.3})
plt.show()

- all right, so now what?

- How can we decide which of the correlated variables to keep?

- We could perform Principal Component Analysis(PCA) on the variables to reduce the dimension.
-> PCA는 StandardScaler 필수! 
-> 알고리즘상 각 feature의 분산이 큰 방향으로 보는 거라서 데이터간 distribution이 다르면 왜곡이 있을 수 있기 때문
- In the All State Claim severity competion, I made this kernel to do that. 
(this kernel: Reducing number of numerical features with PCA)
https://www.kaggle.com/bertcarremans/reducing-number-of-numerical-features-with-pca

# Feature enginnering

- Creating dummy variables

- The values of the categorical variables do not represent any order or magnitude. 
- For instance, category 2 is not twice the value of category 1. 
- Therefore we can create dummy variables to deal with that. 
- We drop the first dummy variable as this information can be derived from the other dummy variables generated for the categories of the original variable.

In [ ]:
# meta: feature 정리
v = meta[(meta.level == 'nominal') & (meta.keep)].index

In [ ]:
train.shape[1]

In [ ]:
v

In [ ]:
train = pd.get_dummies(train, columns=v, drop_first=True)

In [ ]:
train.shape

- Creating interaction variables

In [ ]:
v = meta[(meta.level =='interval') & (meta.keep)].index
v

In [ ]:
poly = PolynomialFeatures(degree = 2, 
                          interaction_only=True, # 자신을 제곱
                          include_bias=False)

In [ ]:
interactions = pd.DataFrame(data = poly.fit_transform(train[v]), columns = poly.get_feature_names(v))
interactions

In [ ]:
interactions.drop(v, axis=1, inplace=True)

In [ ]:
train.shape

In [ ]:
# feature 추가 (axis=1)
train = pd.concat([train, interactions], axis=1)

In [ ]:
# pandas의 모든 함수의 문법은 fit_transform으로 되어있음
poly.fit_transform(train[v])

In [ ]:
poly.get_feature_names(v)

# Feature selection

- Removing features with low or zero variance
- Personally, I prefer to let the classifier algorithm chose which features to keep.
- But there is on thing that we can do ourselves.
- That is removing features with no or a very low variance.
- Sklearn has a handy method to do that: Variance Threshold
(variance가 적으면 feature에서 얻을 정보가 없으므로 삭제 예정)
- By default it removes feature with zero variance.
- This will not be applicable for this competition.
- as we saw there are no zero variance variables in the previous steps.
- But if we would remove features with less than 1% varaince, we would remove 31 variables.

In [ ]:
selector = VarianceThreshold(threshold=0.01)
selector.fit(train.drop(['id', 'target'], axis=1))

In [ ]:
v = train.drop(['id', 'target'], axis=1)

In [ ]:
# True: threshold보다 크다
selector.get_support().sum()
selector.get_support()

In [ ]:
# Function to toggle boolean array elements
# 거꾸로 뒤집음(True -> False)
f = np.vectorize(lambda x : not x)
f(selector.get_support())

In [ ]:
# 결과 뒤집기(같은 방법)
~selector.get_support()

- We would lose rather many variables if we would select based on variance
- But because we do not have so many variables, we'll let the classifier chose. (변수가 많기 때문에 classifier 시킴)
- For data sets with many more variables this could reduce the processing time

- 피처가 너무 많으면 block으로 줄이면서 테스트

## Selecting features with a Random Forest and SelectFromModel
Here we'll base feature selection on the feature importances of a random forest. With Sklearn's SelectFromModel you can then specify how many variables you want to keep. You can set a threshold on the level of feature importance manually. But we'll simply select the top 50% best variables.

In [ ]:
x_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

In [ ]:
feat_labels = x_train.columns
rf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=1)

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
importances = rf.feature_importances_

In [ ]:
# argsort: feature의 importances가 큰 순서대로 index 반
indices = np.argsort(rf.feature_importances_)[::-1]
indices

In [ ]:
for f in range(x_train.shape[1]):
    print('%2d %-*s %f' % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

- With SelectFromModel we can specify which prefit classifier to use and what the threshold is for the feature importances.
- With the get_support method we can then limit the number of variables in the train data.

In [ ]:
# prefit: 이미 학습한 모델을 넘김
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(x_train.shape[1]))
n_features = sfm.transform(x_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train = train[selected_vars + ['target']]

In [ ]:
train.shape

## Feature scaling

- As mentioned before, we can apply standard scaling to the training data. 
- Some classifiers perform better when this is done.

In [ ]:
# StandardScaler: 각 피처의 평균과 분산 정보 저장
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis=1))

### 중요!
scaling으로 train 먼저 fit 시키고 다음에 test에 transform

In [ ]:
scaler = StandardScaler()

df_train = scaler.fit_transform(df_train)
df_test = scaler.transform(df_test)

# Conclusion
Hopefully this notebook helped you with some tips on how to start with this competition. Feel free to vote for it. And if you have questions, post a comment.